In [ ]:
# Code originally by https://github.com/maxjcohen
# My contribution is refine, fix errors and to attempt symbolic representation

Imports

In [1]:
import json
from typing import Optional
import numpy as np
from torch.utils.data import Dataset
import torch
import sys
from ABBA import ABBA as ABBA
import warnings

sys.path.append('C:/Users/rohan/Dissertation/transformer-master/transformer-master')
sys.path.append('C:/Users/rohan/Dissertation/transformer-master/transformer-master/src')
sys.path.append('C:/Users/rohan/Dissertation/transformer-master/transformer-master/dataset')

Access and parse dataset

In [2]:
labels_path: Optional[str] = "labels.json"
normalize: Optional[str] = "max"
_normalize = normalize

# _load_npz(dataset_path, labels_path)
dataset_path = 'dataset/dataset.npz'

dataset = np.load(dataset_path)

with open(labels_path, "r") as stream_json:
    labels = json.load(stream_json)
    
R = dataset['R'].astype(np.float32)
X = dataset['X'].astype(np.float32)
Z = dataset['Z'].astype(np.float32)

m = Z.shape[0]  # Number of training example
K = Z.shape[-1]  # Time serie length

Z = Z.transpose((0, 2, 1))
X = X.transpose((0, 2, 1))

R = np.tile(R[:, np.newaxis, :], (1, K, 1))

Normalization

In [3]:
# Store R, Z and X as x and y
_x = np.concatenate([Z, R], axis=-1)
_y = X

# Normalize
if _normalize == "mean":
    mean = np.mean(_x, axis=(0, 1))
    std = np.std(_x, axis=(0, 1))
    _x = (_x - mean) / (std + np.finfo(float).eps)

    _mean = np.mean(_y, axis=(0, 1))
    _std = np.std(_y, axis=(0, 1))
    _y = (_y - _mean) / (_std + np.finfo(float).eps)
elif _normalize == "max":
    M = np.max(_x, axis=(0, 1))
    m = np.min(_x, axis=(0, 1))
    _x = (_x - m) / (M - m + np.finfo(float).eps)

    _M = np.max(_y, axis=(0, 1))
    _m = np.min(_y, axis=(0, 1))
    _y = (_y - _m) / (_M - _m + np.finfo(float).eps)
elif _normalize is None:
    pass
else:
    raise(
        NameError(f'Normalize method "{_normalize}" not understood.'))

# Convert to float32
_x = torch.Tensor(_x)
_y = torch.Tensor(_y)

Rescaling

In [4]:
def rescale(_normalize,
            y: np.ndarray,
            idx_label: int) -> torch.Tensor:
    """Rescale output from initial normalization.

    Arguments
    ---------
    y:
        Array to resize, of shape (K,).
    idx_label:
        Index of the output label.
    """
    if _normalize == "max":
        return y * (_M[idx_label] - _m[idx_label] + np.finfo(float).eps) + _m[idx_label]
    elif _normalize == "mean":
        return y * (_std[idx_label] + np.finfo(float).eps) + _mean[idx_label]
    else:
        raise(
            NameError(f'Normalize method "{_normalize}" not understood.'))

def __getitem__(idx):
    if torch.is_tensor(idx):
        idx = idx.tolist()
    return (_x[idx], _y[idx])

def __len__(_x):
    return _x.shape[0]

print(_x.size())

torch.Size([7500, 672, 37])


Converting dataset into a version with a windowed time dimension

In [5]:
labels_path: Optional[str] = "labels.json"
window_size: Optional[int] = 5
padding: Optional[int] = 1

print(_x.shape)
print(_y.shape)
    
m, K, d_input = _x.shape
_, _, d_output = _y.shape

step = window_size - 2 * padding
n_step = (K - window_size - 1) // step + 1

dataset_x = np.empty((m, n_step, window_size, d_input), dtype=np.float32)
dataset_y = np.empty((m, n_step, step, d_output), dtype=np.float32)

for idx_step, idx in enumerate(range(0, K-window_size, step)):
    dataset_x[:, idx_step, :] = _x[:, idx:idx+window_size, :]
    dataset_y[:, idx_step, :] = _y[:,idx+padding:idx+window_size-padding, :]

_x = dataset_x
_y = dataset_y

print(_x.size())

TypeError: 'int' object is not callable

Initialize CUDA

In [18]:
torch.cuda.empty_cache()
import gc
gc.collect()
summary = torch.cuda.memory_summary(device=None, abbreviated=False).splitlines()
print(len(summary))
for i in range(0,8):
    print(summary[i])

39
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |


In [19]:
print(type(_x))

<class 'numpy.ndarray'>


Creating a custom dataset class for splitting

In [8]:
class CustomDataset(Dataset):
    
    def __init__(self, x, y, **kwargs):
        super().__init__(**kwargs)
        self._x = x
        self._y = y
    
    def __len__(self):
        return self._x.shape[0]
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return (self._x[idx], self._y[idx])
    

In [12]:
print(type(_x))
print(_x.size())
ozeInstance = CustomDataset(_x, _y)

<class 'numpy.ndarray'>


TypeError: 'int' object is not callable

In [116]:
from torch.utils.data import DataLoader, random_split
dataset_train, dataset_val, dataset_test = random_split(ozeInstance, (5500, 1000, 1000))

In [117]:
print(dataset_train.dataset._x.size())
print(dataset_train.dataset._y.size())

TypeError: 'int' object is not callable

In [74]:
dataset_train.dataset._y[5, :, :].max(-1)[0].size()
print(dataset_train.dataset._x[0:5, :, :].size())
print(dataset_train.dataset._x[5, :, :].size())

torch.Size([5, 672, 37])
torch.Size([672, 37])


Defining LSTM model

In [55]:
class pytorch_LSTM(torch.nn.Module):
    """
    Class to define LSTM model using pytorch.
    """
    def __init__(self, input_dim, hidden_dim, batch_size, output_dim, num_layers, dropout):
        super(pytorch_LSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        self.output_dim = output_dim

        self.states = (0, 0)

        self.lstm = torch.nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, dropout=dropout)

        self.dropout = torch.nn.Dropout(p=dropout)
        self.final = torch.nn.Linear(self.hidden_dim, self.output_dim)
    
    def init_weights(self, m):
        for n, p in m.named_parameters():
            if 'weight_ih' in n:
                for ih in p.chunk(4,0):
                    torch.nn.init.xavier_uniform_(ih)
            elif 'weight_hh' in n:
                for hh in p.chunk(4,0):
                    torch.nn.init.orthogonal_(hh)
            elif 'bias_ih' in n:
                torch.nn.init.zeros_(p)
            elif 'bias_hh' in n:
                torch.nn.init.zeros_(p)
            elif 'final.weight' in n:
                torch.nn.init.xavier_uniform_(p)
            elif 'final.bias' in n:
                torch.nn.init.zeros_(p)

    def initialise_states(self):
        """
        Reset both cell state and hidden state
        """
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim), torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def reset_hidden(self, states):
        """
        Reset hidden state
        """
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim), states[1])

    def reset_cell(self):
        """
        Reset cell state
        """
        return (states[0], torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input, states):
        """
        Define forward pass through LSTM
        """
        # pass through LSTM layers
        lstm_out, states = self.lstm(input.view(len(input), self.batch_size, -1), states)
        # pass through linear layer
        y_pred = self.final(self.dropout(lstm_out[-1].view(self.batch_size, -1)))
        return y_pred.view(-1), states


Building our model

In [98]:
class LSTM_pytorch(object):
    
    def __init__(self, num_layers=2, cells_per_layer=50, dropout=0.5, seed=None, stateful=True, lag=5):
        """
        Initialise and build the model
        """
        self.num_layers = num_layers
        self.cells_per_layer = cells_per_layer
        self.dropout = dropout
        self.seed = seed

        self.stateful = stateful
        self.lag = lag

        if seed != None:
            np.random.seed(seed)
            torch.manual_seed(seed)
        
        
    def build(self, subset, debug=False):
        """
        Build model
        """
        self.features = subset.dataset._x.size()[2]

        self.sequence = subset.dataset

        self.model = pytorch_LSTM(input_dim=self.features, hidden_dim=self.cells_per_layer, batch_size=1, output_dim=self.features, num_layers=self.num_layers, dropout=self.dropout)
        if self.features != 1:
            self.loss_fn = torch.nn.CrossEntropyLoss(reduction='sum')
            self.optimizer = torch.optim.Adam(self.model.parameters())
        else:
            self.loss_fn = torch.nn.MSELoss(size_average=False)
            self.optimizer = torch.optim.Adam(self.model.parameters())
        self.model.init_weights(self.model)


    def construct_training_index(self, debug=False):
        """
        Construct training index (compatible with model) from sequence of vectors of dimension d,
        """
        n = self.sequence._x.size()[2]
        self.index = []
        if self.stateful:
            # Create groups
            self.num_augs = min(self.lag, n - self.lag)
            for el in range(self.num_augs):
                self.index.append(np.arange(el, n - self.lag, self.lag))
        else:
            self.num_augs = 1
            self.index = np.arange(0, n - self.lag, 1)



    def train(self, patience=10, max_epoch=1000, acceptable_loss=np.inf, batch_size=1, weight_restarts=False, debug=False):
        """
        Train the model on the constructed training data
        """
        if batch_size != 1:
            warnings.warn('batch_size must equal 1, setting to 1.')
        ########################################################################
        # Weight restarts
        ########################################################################
        states = self.model.initialise_states()
        if weight_restarts:
            weight_restarts = 10
            store_weights = [0]*weight_restarts
            initial_loss = [0]*weight_restarts
            for i in range(weight_restarts):
                # reset cell state
                states = self.model.initialise_states()

                y_pred, states = self.model(self.sequence._x[0, self.lag, :], (states[0].detach(), states[1].detach()))

                # calculate loss
                if self.features == 1:
                    self.loss = self.loss_fn(y_pred.view(-1, 1), self.sequence._y[0, self.lag, :])
                else:
                    target = self.sequence._y[0, self.lag, :] #.max(-1)[1]
                    self.loss = self.loss_fn(y_pred.reshape(1,-1), target)

                initial_loss[i] = self.loss.data
                store_weights[i] = self.model.state_dict()

                # Re initialise weights
                self.model.init_weights(self.model)
            m = np.argmin(initial_loss)
            self.model.load_state_dict(store_weights[int(m)])
            del store_weights

        ########################################################################
        # Train
        ########################################################################
        vec_loss = np.zeros(max_epoch)
        min_loss = np.inf
        min_loss_ind = np.inf
        losses = [0]*self.num_augs
        if self.stateful: # no shuffle and reset state manually
            for iter in range(max_epoch):
                rint = np.random.permutation(self.num_augs) # shuffle groups
                for r in rint: # run through groups
                    # reset cell state
                    states = self.model.initialise_states()

                    loss_sum = 0
                    for i in self.index[r]: # run through group
                        # Forward pass
                        print(self.sequence._x[0, i, :].size())
                        y_pred, states = self.model(self.sequence._x[0, i, :], (states[0].detach(), states[1].detach()))

                        # calculate loss
                        if self.features == 1:
                            self.loss = self.loss_fn(y_pred.view(-1, 1), self.sequence._y[0, i, :])
                        else:
                            target = self.sequence._y[0, i, :] #.max(-1)[1]
                            self.loss = self.loss_fn(y_pred.reshape(1,-1), target)

                        loss_sum += (float(self.loss.data))**2

                        # Backward pass
                        self.loss.backward(retain_graph=True)

                        # Update parameters
                        self.optimizer.step()
                        # clear gradients
                        self.model.zero_grad()

                    losses[r] = loss_sum/len(self.index[r])
                vec_loss[iter] = np.mean(losses)

                if vec_loss[iter] >= min_loss:
                    if iter - min_loss_ind >= patience and min_loss<acceptable_loss:
                        break
                else:
                    min_loss = vec_loss[iter]
                    old_weights = self.model.state_dict()
                    min_loss_ind = iter

        else: # shuffle in fit
            for iter in range(max_epoch):
                loss_sum = 0
                for i in np.random.permutation(len(self.index)):
                    states = self.model.initialise_states()

                    # Forward pass
                    y_pred, states = self.model.forward(self.sequence._x[0, i, :], (states[0].detach(), states[1].detach()))

                    # calculate loss
                    if self.features == 1:
                        self.loss = self.loss_fn(y_pred.view(-1, 1), self.sequence._y[0, i, :])
                    else:
                        target = self.sequence._y[0, i, :].max(-1)[1]
                        self.loss = self.loss_fn(y_pred.reshape(1,-1), target)

                    loss_sum += (float(self.loss.data))**2

                    # Backward pass
                    self.loss.backward()

                    # Update parameters
                    self.optimizer.step()
                    # clear gradients
                    self.model.zero_grad()

                vec_loss[iter] = loss_sum/len(self.index)

                if vec_loss[iter] >= min_loss:
                    if iter - min_loss_ind >= patience and min_loss < acceptable_loss:
                        break
                else:
                    min_loss = vec_loss[iter]
                    old_weights = self.model.state_dict()
                    min_loss_ind = iter

        self.model.load_state_dict(old_weights)
        self.epoch = iter+1
        self.loss = vec_loss[0:iter+1]

    def forecast(self, k, randomize=False, debug=False):
        """
        Make k step forecast into the future.
        """
        self.model.eval()
        prediction = self.sequence.clone()

        # Recursively make k one-step forecasts
        for ind in range(len(self.sequence), len(self.sequence) + k):
            # Build data to feed into model
            if self.stateful:
                index = np.arange(ind%self.lag, ind, self.lag)
            else:
                index = [ind - self.lag]

            # Feed through model
            states = self.model.initialise_states()
            for i in index:
                p, states = self.model.forward(prediction[0, i, :], (states[0].detach(), states[1].detach()))

            # Convert output
            if self.features != 1:
                softmax = torch.nn.Softmax(dim=-1)
                p = softmax(p).tolist()
                p = np.array(p)
                p /= p.sum()
                if randomize:
                    idx = np.random.choice(range(self.features), p=(p.ravel()))
                else:
                    idx = np.argmax(list(p), axis = 0)

                # Add forecast result to appropriate vectors.
                pred = torch.zeros([1, 1, self.features])
                pred[0, 0, idx] = 1
            else:
                pred = torch.zeros([1, 1, 1])
                pred[0, 0, 0] = p

            prediction = torch.cat([prediction, pred], dim=0)

        if self.features != 1:
            return prediction.view(-1, self.features).tolist()
        else:
            return prediction.view(-1).detach()


Forcasting

In [99]:
model = LSTM_pytorch()
model.build(dataset_train)
model.construct_training_index()

In [100]:
# Train
patience=100
max_epoch=100000
acceptable_loss=np.inf
batch_size=128

model.train(patience=patience, max_epoch=max_epoch, acceptable_loss=acceptable_loss, batch_size=batch_size)

torch.Size([7500, 37])


C:\Users\rohan\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:60: UserWarning: batch_size must equal 1, setting to 1.


RuntimeError: shape '[7500, 7500, -1]' is invalid for input of size 277500